## Altmap Experiments
### Compare altmap to map eq using networkx

In [28]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator)
import numpy as np

# show plots in separate window
%pylab
# load helpers and wrappers
%run helpers.py 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [12]:
# compute essential cost function values for a barbell network with
# given clique size
def ring_of_cliques_cost(clique_size = 3, number_of_cliques = 5, print_output=False):
    nc = clique_size
    c = number_of_cliques
    
    m = (nc*(nc-1) + 2) * c / 2.0 # number of edges in the network
    p0 = (nc - 1) /(2*m) # stat prob for 'normal' nodes
    pc = nc / (2*m) # stat prob for the connecting nodes

    # initial cost
    J_init = c*(nc-1)*p0*np.log2(1.0-p0) + c*pc*np.log2(1.0-pc)
    
    # maximum independent sets cost
    p1 = c*p0; p2 = (c-1)*p0+pc
    J_ind = (nc-1)*p1*np.log2(1.0-p1) + c*p2*np.log2(1.0-p2)
    
    # ground truth cost
    pi = (nc-1)*p0+pc; pnoti=1.0-pi; pinoti=1.0/(2*m); pii = pi - pinoti
    J_true = -c*(pii*np.log2(pii/pi**2) + pinoti*np.log2(pinoti/(pi*pnoti)))
    
    if print_output:
        print (f"\nRing of {c} cliques with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
    
    return J_init, J_true, J_ind

num_cliques = 4
nc_max = 30
nc_list = list(range(3, nc_max + 1))
J_init_list = np.zeros((len(nc_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_list = np.zeros_like(J_init_list)
for i,nc in enumerate(nc_list):
    J_init_list[i], J_true_list[i], J_ind_list[i] = ring_of_cliques_cost(nc, number_of_cliques=num_cliques)


plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# ax.suptitle(f'Ring of {num_cliques} cliques - objective over clique size')
ax.plot(nc_list, -J_true_list, 'x--', label='Ground truth')
ax.plot(nc_list, -J_init_list, '^--', label='Each node as a module')
ax.plot(nc_list, -J_ind_list, 'o--', label='Maximum independet sets')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$', labelpad=20)
ax.set_ylabel('Synthesizing Infomap objective $\mathcal{J}(m)$', labelpad=20)
ax.legend()
plt.tight_layout()

In [13]:
num_cliques = 30
clique_size = 3
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# run community detection
# communities_found, num_communities_found = infomap(G, altmap=False)
# communities_found, num_communities_found = infomap(G, altmap=True)
communities_found, num_communities_found,_,_ = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

# cost = altmap_cost(G, communities_found)
# print (f'Achieved cost L = {cost}')


Spectral clustering finished in 0.08137500000000131 seconds.
OrderedDict([(1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 5), (8, 5), (9, 5), (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 2), (30, 2), (31, 2), (32, 2), (33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 2), (39, 2), (40, 6), (41, 6), (42, 6), (43, 6), (44, 6), (45, 6), (46, 6), (47, 6), (48, 6), (49, 7), (50, 7), (51, 7), (52, 7), (53, 7), (54, 7), (55, 7), (56, 7), (57, 7), (58, 3), (59, 3), (60, 3), (61, 3), (62, 3), (63, 3), (64, 3), (65, 3), (66, 3), (67, 3), (68, 3), (69, 3), (70, 8), (71, 8), (72, 8), (73, 8), (74, 8), (75, 8), (76, 8), (77, 8), (78, 8), (79, 9), (80, 9), (81, 9), (82, 9), (83, 9), (84, 9), (85, 9), (86, 9), (87, 9), (88, 4), (89, 4), (90, 4)])
We found 9 communities.


In [29]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)
plt.tight_layout()

In [10]:
# generate network
num_cliques = 3
clique_size = 4
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * clique_size
for c in range(2, num_cliques+1):
    labels += [c]*clique_size
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = list(range(1,clique_size+1)) * num_cliques
communities_ind = dict(zip(nodes_ids, labels))


plt.close('all')

plt.figure(figsize=(5,8))
drawNetwork(G, communities_true)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_init)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_ind)
plt.tight_layout()

# plt.close('all')
# fig, axs = plt.subplots(1,3)
# fig.suptitle(f'Sample ring of cliques networks')
# 
# drawNetwork(G, communities_true, ax=axs[0])
# axs[0].set_xlabel('Ground truth')
# drawNetwork(G, communities_init, ax=axs[1])
# axs[1].set_xlabel('Initial partition')
# drawNetwork(G, communities_ind, ax=axs[2])
# axs[2].set_xlabel('Maximum independent sets')